# Concept Drift Detection

Steven Sison | April 29, 2024

## Description

This document will be used to set-up and evaluate the concept drift detection algorithm, DDM. The document will include preliminary set-up as well as the necessary evaluations to check the performance of the detector.


## Setting Up the Algorithm

### Steps

1. Generate the Warm-Up Dataset and Streaming Dataset
2. Set-up the Detector Configuration
3. Set-up the metrics such as the error scorers.
4. Start the Concept Drift Detection
    - Warm Up Phase
    - Testing Phase

### Preliminaries

In [1]:
import pandas as pd                     # For data transformation
import numpy as numpy                   # For scientific calculations
import seaborn as sns                   # For data visualizations
import matplotlib.pyplot as plt         # For plotting
import matplotlib.patches as mpatches

# For Concept Drift
from frouros.datasets.synthetic import SEA
from frouros.detectors.concept_drift import DDM, DDMConfig
from frouros.metrics.prequential_error import PrequentialError

import random
from river import drift

### Generating the Warm-Up Dataset and Streaming Dataset

#### Warm-up Dataset

In [2]:
warm_up_dataset = pd.read_csv("binary_new_Bacud_unbalanced_lexical.csv")

warm_up_dataset['actual'] = warm_up_dataset['url_type']
warm_up_dataset['predicted'] = warm_up_dataset['url_type']

warm_up_dataset = warm_up_dataset[['predicted', 'actual']]

warm_up_dataset.head()

,predicted,actual
0,1,1
1,0,0
2,0,0
3,1,1
4,1,1


#### Streaming Dataset

In [29]:
# Temporarily create 4 divisions of the warm-up dataset
# First 10k for warm-up, next 10k for correct predictions, next 10k to induce concept drift

warm_up = warm_up_dataset.head(9500)
temp_after_warm_up = warm_up_dataset[9501: 10001]
temp_concept_drift = warm_up_dataset[10001: 20001]


In [30]:
# Put incorrect predictions in temp_concept_drift
temp_after_warm_up["predicted"] = temp_concept_drift["predicted"].replace({
    0:1,
    1:0,
});

temp_concept_drift["predicted"] = temp_concept_drift["predicted"].replace({
    0:1,
    1:0,
});

temp_concept_drift.head()

C:\Users\sison\AppData\Local\Temp\ipykernel_5092\3158928081.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_after_warm_up["predicted"] = temp_concept_drift["predicted"].replace({
C:\Users\sison\AppData\Local\Temp\ipykernel_5092\3158928081.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_concept_drift["predicted"] = temp_concept_drift["predicted"].replace({


,predicted,actual
10001,1,0
10002,0,1
10003,1,0
10004,1,0
10005,1,0


In [31]:
# Combine the testing phase and with the concept drift

concept_drift_test = pd.concat([warm_up, temp_after_warm_up])

concept_drift_test.head()

,predicted,actual
0,1.0,1
1,0.0,0
2,0.0,0
3,1.0,1
4,1.0,1


### Detector Configuration

In [ ]:
config = DDMConfig(
    warning_level=2.0,
    drift_level=3.0,
    min_num_instances=len(
        warm_up.index
    ),  # Minimum number of instances to start checking for drift
)

detector = DDM(
    config=config,
)

### Error Metrics

In [ ]:
metrics = [
    PrequentialError(
        alpha=alpha,
        name=f"alpha={alpha}",
    )
    for alpha in [1.0, 0.9999, 0.999]
]
metrics_historic_detector = {f"{metric.name}": [] for metric in metrics}


def error_scorer(y_true, y_pred):  # Error function
    return 1 - (y_true == y_pred)

### Concept Drift Detection Using Frouros

### Concept Drift Detection Using Frouros

#### Warm-Up Phase

In [ ]:
# Warm-up detector
print(f"Warming up detector with {len(warm_up.index)} samples")
warm_up_predicted = warm_up['predicted'].tolist()
warm_up_actual = warm_up['actual'].tolist()

for y_pred, y_actual in zip(warm_up_predicted, warm_up_actual):
    error = error_scorer(y_true=y_actual, y_pred=y_pred)
    _ = detector.update(value=error)

    for metric_historic, metric in zip(metrics_historic_detector.keys(), metrics):
        metrics_historic_detector[metric_historic].append(metric(error))

#### Testing Phase

In [ ]:
idx_drift, idx_warning = [], []

i = len(warm_up.index)

test_predicted = testing_phase['predicted'].tolist()
test_actual = testing_phase['actual'].tolist()

for y_pred, y_actual in zip(test_predicted, test_actual):

    error = error_scorer(y_true=y_actual, y_pred=y_pred)
    _ = detector.update(value=error)  # Detector's update

    # All the following lines are optional and only used for plotting the whole process
    for metric_historic, metric in zip(metrics_historic_detector.keys(), metrics):
        metrics_historic_detector[metric_historic].append(metric(error))

    status = detector.status
    if status["drift"]:
        # Drift detected
        print(f"Drift detected at index: {i}")
        idx_drift.append(i)
        detector.reset()  # Reset detector
        for metric in metrics:  # Reset metrics
            metric.reset()
        break  # Stop simulation
    elif status["warning"]:
        # Warning zone
        idx_warning.append(i)
    i += 1

#### Visualization

In [ ]:
plt.rcParams.update({"font.size": 20})

alpha = 0.6
linewidth = 1.0

fig, ax = plt.subplots(
    nrows=2,
    ncols=1,
    figsize=(12, 8),
    sharex=True,
    dpi=300,
    gridspec_kw={"height_ratios": [3, 1]},
)

for (metric_name, metric_values), linecolor in zip(
    metrics_historic_detector.items(),
    ["#1f77b4", "#ff7f0e", "#2ca02c"],
):
    ax[0].plot(
        metric_values,
        color=linecolor,
        linewidth=linewidth,
        alpha=alpha,
        label=metric_name,
    )

drift_color = "red"
drift_linestyle = "--"
warmup_color = "grey"
warning_color = "#e8f519"
real_drift_color = "black"

for idx in range(0, len(warm_up.index)):
    ax[1].axvline(x=idx, color=warmup_color, linewidth=linewidth)

for idx in idx_warning:
    ax[1].axvline(x=idx, color=warning_color, linewidth=linewidth)

for idx in idx_drift:
    ax[1].axvline(x=idx, color=drift_color, linestyle=drift_linestyle, linewidth=1.5)

ax[0].set_ylabel("Prequential Error")
ax[0].legend(
    loc="lower center",
    ncol=3,
    bbox_to_anchor=(0.5, -0.175),
    fancybox=True,
)
ax[1].set_yticks([])
ax[1].set_ylabel("Detector")
drift_path = mpatches.Patch(
    color=drift_color, label="Drift detected", linestyle=drift_linestyle
)
warning_path = mpatches.Patch(color=warning_color, label="Warning")
warmup_path = mpatches.Patch(color=warmup_color, label="Warm up")
real_drift_path = mpatches.Patch(
    color=real_drift_color, label="Real drift", linestyle=drift_linestyle
)
ax[1].axvline(x=20000, color=real_drift_color, linestyle=drift_linestyle, linewidth=1.5)
ax[1].legend(
    handles=[warmup_path, real_drift_path, warning_path, drift_path],
    loc="upper center",
    ncol=4,
    bbox_to_anchor=(0.5, -0.2),
    fancybox=True,
)

fig.tight_layout()
plt.show()

### Concept Drift Detection Using River

In [32]:
concept_drift_test.head()

,predicted,actual
0,1.0,1
1,0.0,0
2,0.0,0
3,1.0,1
4,1.0,1


In [33]:
concept_drift_test['is_correct'] = (concept_drift_test['predicted']==concept_drift_test['actual']).astype(int)

In [34]:
concept_drift_test["is_correct"] = concept_drift_test["is_correct"].replace({
    0:1,
    1:0,
});

concept_drift_test.head(-1)

,predicted,actual,is_correct
0,1.0,1,0
1,0.0,0,0
2,0.0,0,0
3,1.0,1,0
4,1.0,1,0
...,...,...,...
9995,NaN,1,1
9996,NaN,0,1
9997,NaN,1,1
9998,NaN,0,1


In [35]:
ddm = drift.binary.DDM(warm_start=10000)
rng = random.Random(42)

#data_stream = rng.choices([0, 1], k=1000, weights=[0.95, 0.05])
#data_stream = data_stream + rng.choices([0, 1], k=1000, weights=[0.9, 0.1])
#data_stream = data_stream + rng.choices([0, 1], k=1000, weights=[0.3, 0.7])

data_stream = concept_drift_test['is_correct'].tolist()
data_stream = data_stream + rng.choices([0, 1], k=10000, weights=[0.7, 0.3])

print_warning = True
for i, x in enumerate(data_stream):
    ddm.update(x)

    if ddm.warning_detected and print_warning:
        print(f"Warning detected at index {i}")
        print_warning = False
    if ddm.drift_detected:
        print(f"Change detected at index {i}")
        print_warning = True

Warning detected at index 10101
Change detected at index 10198
